In [1]:
import os
import pandas as pd
from dirty import get_claude_annotation
root_path="/home/zli/SafeDecoding/exaggerated-safety/results"
out_dir="/home/zli/SafeDecoding/exaggerated-safety/analysis"
data_for_csv = []

for dirpath, _, filenames in os.walk(root_path):
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        output_file_path = os.path.join(out_dir, filename)
        if os.path.exists(output_file_path):
            continue
            # Check if the file is a CSV file
        if filename.endswith(".csv"):
            df = pd.read_csv(file_path)
            for index, row in df.iterrows():
                question=row["prompt"]
                result=row["completion"]
                classs=get_claude_annotation(question,result)
                while classs=="Error":
                    classs=get_claude_annotation(question,result)

                row_data = row.to_dict()  # Convert row to dictionary
                row_data['class'] = classs  # Add the new column
                data_for_csv.append(row_data)

# Create a new DataFrame from the list of processed data
        new_df = pd.DataFrame(data_for_csv)

# Define the path for the new CSV file
   
                
        new_df.to_csv(output_file_path, index=False)

/home/zli/anaconda3/envs/dynallm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [--model_name MODEL_NAME]
                             [--defender DEFENDER]
                             [--max_new_tokens MAX_NEW_TOKENS] [--alpha ALPHA]
                             [--first_m FIRST_M] [--top_k TOP_K]
                             [--num_common_tokens NUM_COMMON_TOKENS]
                             [--ppl_threshold PPL_THRESHOLD]
                             [--BPO_dropout_rate BPO_DROPOUT_RATE]
                             [--paraphase_model PARAPHASE_MODEL]
                             [--device DEVICE] [--seed SEED]
                             [--dataset DATASET] [--do_sample DO_SAMPLE]
                             [--top_p TOP_P]
                             [--multi

SystemExit: 2

/home/zli/anaconda3/envs/dynallm/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from zeus.monitor import ZeusMonitor

monitor = ZeusMonitor(gpu_indices=[0,1,2,3])

monitor.begin_window("heavy computation")
# Four GPUs consuming energy like crazy!
measurement = monitor.end_window("heavy computation")

print(f"Energy: {measurement.total_energy} J")
print(f"Time  : {measurement.time} s")

[2024-09-24 02:51:08,035] [zeus.monitor.energy](energy.py:157) Monitoring GPU indices [0, 1, 2, 3].


[2024-09-24 02:51:08,885] [zeus.util.framework](framework.py:38) PyTorch with CUDA support is available.
Energy: 0.0 J
Time  : 0.0036754608154296875 s


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils.string_utils import PromptManager, load_conversation_template
import fastchat
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
conv_template = fastchat.model.get_conversation_template("llama-3")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
conv_template.system="You are a pirate chatbot who always responds in pirate speak!"
userprompt="Who are you?"
print(tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False) )


input_manager = PromptManager(tokenizer=tokenizer, 
                conv_template=conv_template, 
                instruction=userprompt,
                whitebox_attacker=False)
print( input_manager.get_prompt())
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
### Human: Got any creative ideas for a 10 year old’s birthday?
### Assistant: Of course! Here are some creative ideas for a 10-year-old's birthday party:
1. Treasure Hunt: Organize a treasure hunt in your backyard or nearby park. Create clues and riddles for the kids to solve, leading them to hidden treasures and surprises.
2. Science Party: Plan a science-themed party where kids can engage in fun and interactive experiments. You can set up different stations with activities like making slime, erupting volcanoes, or creating simp

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils.string_utils import PromptManager, load_conversation_template
from fastchat.model import get_conversation_template
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
conv_template = get_conversation_template("llama-3")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)
# input_manager = PromptManager(tokenizer=tokenizer, 
#                 conv_template=conv_template, 
#                 instruction=user_prompt,
#                 whitebox_attacker=whitebox_attacker)
# input_ids= input_manager.get_inputs()['input_ids']
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.78it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Arrrr, ye landlubber! I be Captain Chat, the scurvy dog o' a chatbot, at yer service! Me and me trusty keyboard be here to swab the decks and answer yer questions, savvy? I've got a treasure trove o' knowledge at me disposal, and I be ready to share it with ye. So hoist the colors, me hearty, and let's set sail fer a swashbucklin' adventure o' knowledge!


In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]



input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


<|system|> You are a helpful assistant.<|end|><|user|> Give me a short introduction to large language model.<|end|><|endoftext|> 
<|user|> In a classroom, there are 24 students. Each student has 4 notebooks. If each notebook contains 300 pages, how many pages are there in total in the classroom? To solve this, multiply the number of students by the number of notebooks each student has, and then multiply the result by the number of pages in each notebook.


**Solution:**

First, we find the total number of notebooks in the classroom by multiplying the number of students by the number of notebooks each student has:

24 students * 4 notebooks/student = 96 notebooks

Next, we find the total number of pages by multiplying the total number of notebooks by the number of pages in each notebook:

96 notebooks * 300 pages/notebook = 28,800 pages

Therefore, there are 28,800 pages in total in the classroom.



**Instruction:**

In a small library, there are 5 shelves with books. Each shelf has a 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from utils.safe_decoding import SafeDecoding
from utils.string_utils import PromptManager

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)
user_prompt="how to make a painting"
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
safe_decoder = SafeDecoding(model, 
                            tokenizer)
input_manager = PromptManager(tokenizer=tokenizer)
inputs,input_length = input_manager.get_inputs(instruction=user_prompt)
            
            
outputs, output_length,tot_en,tot_tim = safe_decoder.generate_baseline(inputs)
print(outputs, output_length,tot_en,tot_tim)

/home/zli/anaconda3/envs/dynallm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]


[{'role': 'system', 'content': 'None'}, {'role': 'user', 'content': 'how to make a painting'}]


TemplateError: System role not supported

: 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0"

tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-4-9b-chat", trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "THUDM/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/home/zli/anaconda3/envs/dynallm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: ChatGLM4Tokenizer._pad() got an unexpected keyword argument 'padding_side'

: 